# 📊 Consultas MDX Básicas

**Objetivo**: Dominar la sintaxis fundamental de consultas MDX en Atoti

## 🎯 Lo que aprenderás
- 🔰 Sintaxis básica de consultas MDX
- 🔰 Filtros y agrupaciones simples
- 🚀 Consultas bi-dimensionales
- 🚀 Ordenamiento y limitación de resultados

## 📋 Prerrequisitos
- ✅ Conceptos básicos de dimensiones y medidas
- ✅ Conexión a Oracle disponible

---

**💡 NOTA**: Este notebook recrea el cubo desde cero para garantizar compatibilidad entre sesiones.

## 🔧 Configuración y Recreación del Cubo

In [10]:
# Importar librerías necesarias
import atoti as tt
from atoti_jdbc import JdbcLoad
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

# Configuración de conexión a Oracle
ORACLE_USER = os.getenv('ORACLE_USER', 'C##DM_ACADEMICO')
ORACLE_PASSWORD = os.getenv('ORACLE_PASSWORD', 'YourPassword123')
ORACLE_HOST = os.getenv('ORACLE_HOST', 'localhost')
ORACLE_PORT = os.getenv('ORACLE_PORT', '1521')
ORACLE_SERVICE = os.getenv('ORACLE_SERVICE', 'XEPDB1')

jdbc_url = f"jdbc:oracle:thin:{ORACLE_USER}/{ORACLE_PASSWORD}@//{ORACLE_HOST}:{ORACLE_PORT}/{ORACLE_SERVICE}"

print("✅ Configuración completada")
print(f"🔗 Conectando a: {ORACLE_HOST}:{ORACLE_PORT}/{ORACLE_SERVICE}")

✅ Configuración completada
🔗 Conectando a: localhost:1521/XEPDB1


In [11]:
# Cerrar sesiones previas e iniciar nueva sesión
try:
    if 'session' in globals():
        session.close()
        print("✅ Sesión anterior cerrada")
except:
    pass

# Iniciar nueva sesión
session = tt.Session.start()
print("🚀 Sesión de Atoti iniciada")
print(f"📍 URL del servidor: {session.url}")

✅ Sesión anterior cerrada
🚀 Sesión de Atoti iniciada
📍 URL del servidor: http://localhost:52614


In [12]:
# 🔰 FUNCIÓN AUXILIAR: Cargar tablas de forma segura
def cargar_tabla_academica(nombre_tabla, query_sql, claves_primarias):
    """
    Carga una tabla desde Oracle con validaciones.
    """
    try:
        jdbc_load = JdbcLoad(query_sql, url=jdbc_url)
        data_types = session.tables.infer_data_types(jdbc_load)
        
        tabla = session.create_table(
            nombre_tabla, 
            data_types=data_types, 
            keys=claves_primarias
        )
        
        tabla.load(jdbc_load)
        print(f"✅ Tabla {nombre_tabla} cargada exitosamente")
        return tabla
        
    except Exception as e:
        print(f"❌ Error cargando {nombre_tabla}: {str(e)}")
        return None

print("🛠️ Función auxiliar configurada")

🛠️ Función auxiliar configurada


In [13]:
# 🔰 RECREAR CUBO ACADÉMICO BÁSICO
print("📚 Recreando cubo académico para consultas MDX...")

# Cargar tabla de hechos principal
tabla_matriculas = cargar_tabla_academica(
    "Matriculas",
    "SELECT * FROM F_MATRICULA WHERE ROWNUM <= 500",  # Dataset manejable
    {"ID_ALUMNO", "ID_CURSO_ACADEMICO", "ID_ASIGNATURA"}
)

# Cargar dimensiones esenciales
tabla_cursos = cargar_tabla_academica(
    "CursosAcademicos",
    "SELECT * FROM D_CURSO_ACADEMICO",
    {"ID_CURSO_ACADEMICO"}
)

tabla_sexo = cargar_tabla_academica(
    "Sexo", 
    "SELECT * FROM D_SEXO",
    {"ID_SEXO"}
)

if tabla_matriculas and tabla_cursos and tabla_sexo:
    print("\n🎉 Tablas básicas cargadas correctamente")
else:
    print("\n⚠️ Error en carga de tablas - verificar conexión")

📚 Recreando cubo académico para consultas MDX...
✅ Tabla Matriculas cargada exitosamente
✅ Tabla CursosAcademicos cargada exitosamente
✅ Tabla Sexo cargada exitosamente

🎉 Tablas básicas cargadas correctamente


In [14]:
# 🔰 CREAR CUBO Y CONFIGURAR UNIONES
if tabla_matriculas:
    # Establecer uniones
    if tabla_cursos:
        tabla_matriculas.join(
            tabla_cursos,
            tabla_matriculas["ID_CURSO_ACADEMICO"] == tabla_cursos["ID_CURSO_ACADEMICO"]
        )
        print("🔗 Unión con cursos académicos establecida")
    
    if tabla_sexo:
        tabla_matriculas.join(
            tabla_sexo,
            tabla_matriculas["ID_SEXO"] == tabla_sexo["ID_SEXO"]
        )
        print("🔗 Unión con dimensión sexo establecida")
    
    # Crear el cubo
    cubo = session.create_cube(tabla_matriculas, "CuboMDX")
    
    # Referencias de acceso rápido
    h = cubo.hierarchies  # Jerarquías
    l = cubo.levels      # Niveles
    m = cubo.measures    # Medidas
    
    print("\n🧊 Cubo MDX creado exitosamente")
    print(f"📊 Jerarquías disponibles: {len(h)}")
    print(f"📈 Medidas automáticas: {len(m)}")
    print(f"🎯 Niveles para agrupar: {len(l)}")

🔗 Unión con cursos académicos establecida
🔗 Unión con dimensión sexo establecida

🧊 Cubo MDX creado exitosamente
📊 Jerarquías disponibles: 37
📈 Medidas automáticas: 134
🎯 Niveles para agrupar: 37


## 🔰 FUNDAMENTOS: Anatomía de una Consulta MDX

### La Estructura Básica
```python
resultado = cubo.query(
    medidas,        # ¿Qué queremos medir?
    levels=niveles, # ¿Cómo queremos agrupar?
    filter=filtro   # ¿Qué datos incluir? (opcional)
)
```

### 🧭 Conceptos Clave
- **Medidas**: Valores numéricos a calcular (ej: conteos, promedios)
- **Niveles**: Dimensiones para agrupar datos (ej: por semestre, carrera)
- **Filtros**: Condiciones para limitar los datos analizados

In [15]:
# 🔰 EXPLORAR ESTRUCTURA DEL CUBO
print("🔍 EXPLORANDO NUESTRO CUBO PARA CONSULTAS MDX:")

# Función auxiliar para explicar consultas
def explicar_consulta(descripcion, concepto, resultado):
    """Patrón estándar para explicar cada consulta MDX"""
    print(f"\n🔍 {descripcion}")
    print(f"💡 Concepto: {concepto}")
    print("✅ Resultado:")
    display(resultado)
    print(f"📊 Filas obtenidas: {len(resultado)}")
    print("-" * 50)

# Mostrar medidas principales
print("\n📈 MEDIDAS PRINCIPALES:")
medidas_principales = [k for k in m.keys() if 'COUNT' in k.upper()][:3]
for medida in medidas_principales:
    print(f"  📊 {medida}")

# Mostrar niveles principales  
print("\n📐 NIVELES PRINCIPALES:")
niveles_principales = [k for k in l.keys()][:5]
for nivel in niveles_principales:
    print(f"  🎯 {nivel}")

print("\n🎉 Cubo listo para consultas MDX básicas")

🔍 EXPLORANDO NUESTRO CUBO PARA CONSULTAS MDX:

📈 MEDIDAS PRINCIPALES:
  📊 contributors.COUNT

📐 NIVELES PRINCIPALES:
  🎯 ('CursosAcademicos', 'NOMBRE_CURSO_ACADEMICO', 'NOMBRE_CURSO_ACADEMICO')
  🎯 ('Sexo', 'FECHA_CARGA', 'FECHA_CARGA')
  🎯 ('Sexo', 'NOMBRE_SEXO', 'NOMBRE_SEXO')
  🎯 ('Sexo', 'ID_SEXO_NK', 'ID_SEXO_NK')
  🎯 ('Matriculas', 'SN_DISCAPACIDAD', 'SN_DISCAPACIDAD')

🎉 Cubo listo para consultas MDX básicas


## 🔰 SECCIÓN 1: Consultas con Una Dimensión

In [16]:
# 🔰 CONSULTA BÁSICA 1: Total general (sin agrupación)
print("🔍 CONSULTA 1: ¿Cuántos registros tenemos en total?")
print("💡 Concepto: Consulta MDX más simple - solo medida, sin agrupación")

# Seleccionar primera medida de conteo disponible
medida_count = next(iter([k for k in m.keys() if 'COUNT' in k.upper()]), 'contributors.COUNT')

resultado_total = cubo.query(m[medida_count])

explicar_consulta(
    "CONSULTA 1: Total general de registros",
    "Consulta base: una medida sin dimensiones = resultado global",
    resultado_total
)

🔍 CONSULTA 1: ¿Cuántos registros tenemos en total?
💡 Concepto: Consulta MDX más simple - solo medida, sin agrupación

🔍 CONSULTA 1: Total general de registros
💡 Concepto: Consulta base: una medida sin dimensiones = resultado global
✅ Resultado:


,contributors.COUNT
0,500


📊 Filas obtenidas: 1
--------------------------------------------------


In [17]:
# 🔰 CONSULTA BÁSICA 2: Agrupación por una dimensión
print("🔍 CONSULTA 2: Distribución por semestre")
print("💡 Concepto: Agrupación simple - medida + una dimensión")

# Buscar nivel de semestre disponible
nivel_semestre = None
for nivel_key in l.keys():
    if 'SEMESTRE' in str(nivel_key).upper():
        nivel_semestre = l[nivel_key]
        print(f"🎯 Usando nivel: {nivel_key}")
        break

if nivel_semestre:
    resultado_semestre = cubo.query(
        m[medida_count],
        levels=[nivel_semestre]
    )
    
    explicar_consulta(
        "CONSULTA 2: Registros por semestre",
        "Agrupación básica: cada fila = un semestre, valor = conteo",
        resultado_semestre
    )
    
    # Análisis rápido
    print(f"📊 Semestres encontrados: {len(resultado_semestre)}")
    if len(resultado_semestre) > 0:
        print(f"📈 Total registros: {resultado_semestre.sum().iloc[0]:,.0f}")
else:
    print("⚠️ No se encontró nivel SEMESTRE, usando primer nivel disponible")
    primer_nivel = l[list(l.keys())[0]]
    
    resultado_alternativo = cubo.query(
        m[medida_count],
        levels=[primer_nivel]
    )
    
    explicar_consulta(
        f"CONSULTA 2 (alternativa): Agrupación por {list(l.keys())[0]}",
        "Adaptación: usando primer nivel disponible cuando el ideal no existe",
        resultado_alternativo.head()
    )

🔍 CONSULTA 2: Distribución por semestre
💡 Concepto: Agrupación simple - medida + una dimensión
⚠️ No se encontró nivel SEMESTRE, usando primer nivel disponible

🔍 CONSULTA 2 (alternativa): Agrupación por ('CursosAcademicos', 'NOMBRE_CURSO_ACADEMICO', 'NOMBRE_CURSO_ACADEMICO')
💡 Concepto: Adaptación: usando primer nivel disponible cuando el ideal no existe
✅ Resultado:


,contributors.COUNT
NOMBRE_CURSO_ACADEMICO,
2010/2011,28
2011/2012,31
2012/2013,38
2013/2014,50
2014/2015,27


📊 Filas obtenidas: 5
--------------------------------------------------


## 🔰→🚀 SECCIÓN 2: Múltiples Medidas

In [18]:
# 🚀 MÚLTIPLES MEDIDAS: Análisis completo
print("🔍 CONSULTA 3: Análisis con múltiples medidas")
print("💡 Concepto: Combinar varias métricas en una sola consulta")

# Seleccionar múltiples medidas disponibles
medidas_disponibles = list(m.keys())
medidas_seleccionadas = medidas_disponibles[:3]  # Primeras 3 medidas

print(f"📊 Medidas seleccionadas: {medidas_seleccionadas}")

# Usar nivel que ya sabemos que funciona
nivel_para_agrupar = nivel_semestre if nivel_semestre else l[list(l.keys())[0]]

resultado_multiple = cubo.query(
    *[m[medida] for medida in medidas_seleccionadas],
    levels=[nivel_para_agrupar]
)

explicar_consulta(
    "CONSULTA 3: Análisis multi-medida",
    "Vista panorámica: múltiples métricas por grupo",
    resultado_multiple
)

print("🚀 ANÁLISIS AVANZADO:")
print("  - Cada fila representa un grupo de la dimensión")
print("  - Cada columna representa una medida diferente")
print("  - Permite comparaciones multivariadas")

🔍 CONSULTA 3: Análisis con múltiples medidas
💡 Concepto: Combinar varias métricas en una sola consulta
📊 Medidas seleccionadas: ['NUM_REGS_GRANO_MOV.MEAN', 'ID_CAMPUS_CENTRO.SUM', 'FLG_TRASLADO_MISMO_ESTUDIO.MEAN']

🔍 CONSULTA 3: Análisis multi-medida
💡 Concepto: Vista panorámica: múltiples métricas por grupo
✅ Resultado:


,NUM_REGS_GRANO_MOV.MEAN,ID_CAMPUS_CENTRO.SUM,FLG_TRASLADO_MISMO_ESTUDIO.MEAN
NOMBRE_CURSO_ACADEMICO,,,
2010/2011,,,.43
2011/2012,,,.55
2012/2013,,,.47
2013/2014,,,.40
2014/2015,,,.44
2015/2016,,,.39
2016/2017,,,.50
2017/2018,,,.59
2018/2019,,,.43


📊 Filas obtenidas: 15
--------------------------------------------------
🚀 ANÁLISIS AVANZADO:
  - Cada fila representa un grupo de la dimensión
  - Cada columna representa una medida diferente
  - Permite comparaciones multivariadas


## 🔰→🚀 SECCIÓN 3: Filtros Básicos

In [19]:
# 🔰 FILTROS SIMPLES
print("🔍 CONSULTA 4: Aplicando filtros básicos")
print("💡 Concepto: Incluir solo datos que cumplan condiciones específicas")

# Primero, explorar valores disponibles para filtrar
if nivel_semestre:
    # Obtener muestra de datos para ver valores disponibles
    muestra_datos = cubo.query(
        m[medida_count],
        levels=[nivel_semestre]
    )
    
    print("📋 Valores disponibles para filtrar:")
    valores_disponibles = muestra_datos.index.tolist()[:3]
    for valor in valores_disponibles:
        print(f"  📌 {valor}")
    
    # Aplicar filtro con primer valor disponible
    if valores_disponibles:
        valor_filtro = valores_disponibles[0]
        
        resultado_filtrado = cubo.query(
            m[medida_count],
            levels=[nivel_semestre],
            filter=nivel_semestre == valor_filtro
        )
        
        explicar_consulta(
            f"CONSULTA 4: Solo datos de '{valor_filtro}'",
            "Filtro de igualdad: condición simple sobre dimensión",
            resultado_filtrado
        )
    else:
        print("⚠️ No se encontraron valores para filtrar")
else:
    print("⚠️ Usando filtro alternativo con medidas numéricas")
    
    # Filtro numérico sobre medidas
    resultado_filtrado_num = cubo.query(
        m[medida_count],
        levels=[nivel_para_agrupar],
        filter=m[medida_count] > 1  # Solo grupos con más de 1 registro
    )
    
    explicar_consulta(
        "CONSULTA 4: Filtro numérico (conteo > 1)",
        "Filtro sobre medida: excluir grupos pequeños",
        resultado_filtrado_num
    )

🔍 CONSULTA 4: Aplicando filtros básicos
💡 Concepto: Incluir solo datos que cumplan condiciones específicas
⚠️ Usando filtro alternativo con medidas numéricas

🔍 CONSULTA 4: Filtro numérico (conteo > 1)
💡 Concepto: Filtro sobre medida: excluir grupos pequeños
✅ Resultado:


,contributors.COUNT
NOMBRE_CURSO_ACADEMICO,
2010/2011,28
2011/2012,31
2012/2013,38
2013/2014,50
2014/2015,27
2015/2016,36
2016/2017,28
2017/2018,29
2018/2019,30


📊 Filas obtenidas: 15
--------------------------------------------------


In [20]:
# 🚀 FILTROS COMPLEJOS
print("🚀 FILTROS AVANZADOS: Condiciones múltiples")
print("💡 Concepto: Combinar múltiples condiciones con operadores lógicos")

try:
    # Filtro compuesto: combinar condiciones con &
    resultado_complejo = cubo.query(
        m[medida_count],
        *[m[medida] for medida in medidas_seleccionadas[:2]],
        levels=[nivel_para_agrupar],
        filter=(
            (m[medida_count] >= 1) &  # Condición numérica
            (m[medidas_seleccionadas[0]] >= 0)  # Segunda condición
        )
    )
    
    explicar_consulta(
        "CONSULTA 5: Filtro compuesto (múltiples condiciones)",
        "Lógica compleja: condición1 AND condición2",
        resultado_complejo
    )
    
    print("💡 Operadores de filtro disponibles:")
    print("  - Igualdad: ==  │  Desigualdad: !=  │  Mayor: >  │  Menor: <")
    print("  - Y lógico: &   │  O lógico: |      │  Negación: ~")
    
except Exception as e:
    print(f"⚠️ Error en filtro complejo: {e}")
    print("💡 Los filtros complejos requieren compatibilidad de tipos")

🚀 FILTROS AVANZADOS: Condiciones múltiples
💡 Concepto: Combinar múltiples condiciones con operadores lógicos

🔍 CONSULTA 5: Filtro compuesto (múltiples condiciones)
💡 Concepto: Lógica compleja: condición1 AND condición2
✅ Resultado:


,contributors.COUNT,NUM_REGS_GRANO_MOV.MEAN,ID_CAMPUS_CENTRO.SUM
NOMBRE_CURSO_ACADEMICO,,,
2010/2011,28,,
2011/2012,31,,
2012/2013,38,,
2013/2014,50,,
2014/2015,27,,
2015/2016,36,,
2016/2017,28,,
2017/2018,29,,
2018/2019,30,,


📊 Filas obtenidas: 15
--------------------------------------------------
💡 Operadores de filtro disponibles:
  - Igualdad: ==  │  Desigualdad: !=  │  Mayor: >  │  Menor: <
  - Y lógico: &   │  O lógico: |      │  Negación: ~


## 🚀 SECCIÓN 4: Consultas Bi-dimensionales

In [23]:
# 🚀 DOS DIMENSIONES: Análisis cruzado
print("🔍 CONSULTA 6: Análisis con dos dimensiones")
print("💡 Concepto: Tabla dinámica - cruce de dos variables categóricas")

# Buscar dimensiones académicas relevantes - CORRECCIÓN ESPECÍFICA
primer_nivel = None
segundo_nivel = None

# Buscar NOMBRE_SEXO específicamente
for nivel_key in l.keys():
    if 'NOMBRE_SEXO' in str(nivel_key).upper():
        primer_nivel = l[nivel_key]
        primer_nivel_key = nivel_key
        print(f"🎯 Primera dimensión encontrada: {nivel_key}")
        break

# Buscar segunda dimensión relevante (no FECHA_CARGA)
for nivel_key in l.keys():
    if ('CURSO' in str(nivel_key).upper() or 
        'SEMESTRE' in str(nivel_key).upper() or
        'ASIGNATURA' in str(nivel_key).upper()) and nivel_key != primer_nivel_key:
        segundo_nivel = l[nivel_key]
        segundo_nivel_key = nivel_key
        print(f"🎯 Segunda dimensión encontrada: {nivel_key}")
        break

# Validar que encontramos ambas dimensiones
if primer_nivel is not None and segundo_nivel is not None:
    try:
        resultado_bidimensional = cubo.query(
            m[medida_count],
            levels=[primer_nivel, segundo_nivel],
            include_totals=True
        )
        
        explicar_consulta(
            f"CONSULTA 6: Análisis {primer_nivel_key} × {segundo_nivel_key}",
            "Análisis cruzado: intersección de dimensiones académicas relevantes",
            resultado_bidimensional.head(10)
        )
        
        print("🔍 Interpretación académica específica:")
        print(f"  - Filas: Diferentes valores de {primer_nivel_key}")
        print(f"  - Columnas: Diferentes valores de {segundo_nivel_key}")
        print("  - Celdas: Número de registros en esa combinación")
        print("  - Análisis útil para detectar distribuciones académicas")
        
    except Exception as e:
        print(f"⚠️ Error en consulta bidimensional: {e}")
        print("🔄 Usando análisis alternativo...")
        
        # Alternativa: solo NOMBRE_SEXO si está disponible
        if primer_nivel is not None:
            resultado_sexo = cubo.query(
                m[medida_count],
                levels=[primer_nivel]
            )
            
            explicar_consulta(
                "CONSULTA 6: Distribución por sexo (alternativa)",
                "Análisis unidimensional: estudiantes por género",
                resultado_sexo
            )

elif primer_nivel is not None:
    # Solo tenemos NOMBRE_SEXO
    resultado_sexo = cubo.query(
        m[medida_count],
        levels=[primer_nivel]
    )
    
    explicar_consulta(
        "CONSULTA 6: Distribución por sexo",
        "Análisis por género: base para análisis demográfico",
        resultado_sexo
    )
    
    print("💡 ANÁLISIS DEMOGRÁFICO:")
    print("  - Distribución de género en la institución")
    print("  - Base para políticas de equidad")
    print("  - Útil para planificación de recursos")

else:
    print("⚠️ No se encontró NOMBRE_SEXO - usando niveles disponibles")
    
    # Fallback a análisis con múltiples medidas
    resultado_multi_medida = cubo.query(
        *[m[medida] for medida in medidas_seleccionadas[:2]],
        levels=[nivel_para_agrupar]
    )
    
    explicar_consulta(
        "CONSULTA 6: Múltiples medidas por dimensión",
        "Análisis multivariado como alternativa",
        resultado_multi_medida
    )

# 💡 EXPLICACIÓN CONTEXTUAL 
print("\n💡 CONTEXTO ACADÉMICO RELEVANTE:")
print("  - NOMBRE_SEXO: Análisis demográfico (Masculino/Femenino)")
print("  - CURSO_ACADEMICO: Dimensión temporal para tendencias")
print("  - SEMESTRE: Análisis por períodos académicos")
print("  - ASIGNATURA: Distribución por materias")
print("  - ❌ FECHA_CARGA: No relevante para análisis académico")

🔍 CONSULTA 6: Análisis con dos dimensiones
💡 Concepto: Tabla dinámica - cruce de dos variables categóricas
🎯 Primera dimensión encontrada: ('Sexo', 'NOMBRE_SEXO', 'NOMBRE_SEXO')
🎯 Segunda dimensión encontrada: ('CursosAcademicos', 'NOMBRE_CURSO_ACADEMICO', 'NOMBRE_CURSO_ACADEMICO')

🔍 CONSULTA 6: Análisis ('Sexo', 'NOMBRE_SEXO', 'NOMBRE_SEXO') × ('CursosAcademicos', 'NOMBRE_CURSO_ACADEMICO', 'NOMBRE_CURSO_ACADEMICO')
💡 Concepto: Análisis cruzado: intersección de dimensiones académicas relevantes
✅ Resultado:


contributors.COUNT
NOMBRE_SEXO NOMBRE_CURSO_ACADEMICO                    
<NA>        <NA>                                   500
Hombre      <NA>                                   232
            2010/2011                               11
            2011/2012                               12
            2012/2013                               18
            2013/2014                               25
            2014/2015                               11
            2015/2016                               17
            2016/2017                               14
            2017/2018                               10

📊 Filas obtenidas: 10
--------------------------------------------------
🔍 Interpretación académica específica:
  - Filas: Diferentes valores de ('Sexo', 'NOMBRE_SEXO', 'NOMBRE_SEXO')
  - Columnas: Diferentes valores de ('CursosAcademicos', 'NOMBRE_CURSO_ACADEMICO', 'NOMBRE_CURSO_ACADEMICO')
  - Celdas: Número de registros en esa combinación
  - Análisis útil para detectar distribuciones académicas

💡 CONTEXTO ACADÉMICO RELEVANTE:
  - NOMBRE_SEXO: Análisis demográfico (Masculino/Femenino)
  - CURSO_ACADEMICO: Dimensión temporal para tendencias
  - SEMESTRE: Análisis por períodos académicos
  - ASIGNATURA: Distribución por materias
  - ❌ FECHA_CARGA: No relevante para análisis académico


## 🚀 SECCIÓN 5: Ordenamiento y Limitación

In [24]:
# 🚀 ORDENAMIENTO Y TOP N
print("🔍 CONSULTA 7: Top N y ordenamiento de resultados")
print("💡 Concepto: Combinar consulta MDX + manipulación pandas")

# Obtener datos para ordenar
datos_para_ordenar = cubo.query(
    m[medida_count],
    levels=[nivel_para_agrupar]
)

# Aplicar ordenamiento descendente
top_5 = datos_para_ordenar.sort_values(
    by=datos_para_ordenar.columns[0],  # Ordenar por primera columna
    ascending=False
).head(5)

explicar_consulta(
    "CONSULTA 7: Top 5 por conteo (descendente)",
    "Pipeline: Consulta MDX → Ordenamiento pandas → Limitación",
    top_5
)

print("💡 Patrón de ordenamiento en MDX:")
print("  1. 🔍 Ejecutar consulta MDX completa")
print("  2. 📊 Aplicar .sort_values() con pandas")
print("  3. 🎯 Usar .head(n) o .tail(n) para limitar")
print("  4. 📈 Analizar patrones en los extremos")

🔍 CONSULTA 7: Top N y ordenamiento de resultados
💡 Concepto: Combinar consulta MDX + manipulación pandas

🔍 CONSULTA 7: Top 5 por conteo (descendente)
💡 Concepto: Pipeline: Consulta MDX → Ordenamiento pandas → Limitación
✅ Resultado:


,contributors.COUNT
NOMBRE_CURSO_ACADEMICO,
2013/2014,50
2012/2013,38
2024/2025,38
2022/2023,38
2015/2016,36


📊 Filas obtenidas: 5
--------------------------------------------------
💡 Patrón de ordenamiento en MDX:
  1. 🔍 Ejecutar consulta MDX completa
  2. 📊 Aplicar .sort_values() con pandas
  3. 🎯 Usar .head(n) o .tail(n) para limitar
  4. 📈 Analizar patrones en los extremos


In [25]:
# 🚀 ANÁLISIS COMPARATIVO: Top vs Bottom
print("🔍 CONSULTA 8: Análisis comparativo de extremos")

# Top 3 y Bottom 3
top_3 = datos_para_ordenar.sort_values(
    by=datos_para_ordenar.columns[0], 
    ascending=False
).head(3)

bottom_3 = datos_para_ordenar.sort_values(
    by=datos_para_ordenar.columns[0], 
    ascending=True
).head(3)

print("🏆 TOP 3 (valores más altos):")
display(top_3)

print("\n📉 BOTTOM 3 (valores más bajos):")
display(bottom_3)

# Estadísticas descriptivas
if len(datos_para_ordenar) > 0:
    columna = datos_para_ordenar.columns[0]
    promedio = datos_para_ordenar[columna].mean()
    maximo = datos_para_ordenar[columna].max()
    minimo = datos_para_ordenar[columna].min()
    
    print(f"\n📊 ESTADÍSTICAS DESCRIPTIVAS:")
    print(f"  - 📈 Promedio: {promedio:.1f}")
    print(f"  - 🔝 Máximo: {maximo:,.0f} ({top_3.index[0]})")
    print(f"  - 🔻 Mínimo: {minimo:,.0f} ({bottom_3.index[0]})")
    print(f"  - 📊 Total registros analizados: {len(datos_para_ordenar)}")

🔍 CONSULTA 8: Análisis comparativo de extremos
🏆 TOP 3 (valores más altos):


,contributors.COUNT
NOMBRE_CURSO_ACADEMICO,
2013/2014,50
2012/2013,38
2024/2025,38



📉 BOTTOM 3 (valores más bajos):


,contributors.COUNT
NOMBRE_CURSO_ACADEMICO,
2014/2015,27
2010/2011,28
2016/2017,28



📊 ESTADÍSTICAS DESCRIPTIVAS:
  - 📈 Promedio: 33.3
  - 🔝 Máximo: 50 (2013/2014)
  - 🔻 Mínimo: 27 (2014/2015)
  - 📊 Total registros analizados: 15


## 🎯 EJERCICIOS PRÁCTICOS

### 🔰 Ejercicios Básicos

In [39]:
# 🔰 EJERCICIO 1: Tu primera consulta personalizada
print("🎯 EJERCICIO 1: Crea una consulta que agrupe por una dimensión diferente")
print("💪 Objetivo: Practicar la sintaxis básica de consulta MDX")
print("📝 Instrucciones:")
print("   1. Selecciona una medida disponible")
print("   2. Selecciona un nivel para agrupar (diferente a los ejemplos)")
print("   3. Ejecuta la consulta")
print("   4. Interpreta los resultados")

print("\n📋 Recursos disponibles:")
print(f"  Medidas: {list(m.keys())[:3]}...")
print(f"  Niveles: {list(l.keys())[:3]}...")

print("\n💡 Pista: ejercicio_1 = cubo.query(m['tu_medida'], levels=[l['tu_nivel']])")
print("📝 Escribe tu código aquí:\n")

# TU CÓDIGO AQUÍ:
# ejercicio_1 = cubo.query(...)
# display(ejercicio_1)

# LÍNEA DIVISORIA - SOLUCIÓN ABAJO
print("\n" + "="*50)
print("✅ EJEMPLO DE SOLUCIÓN:")

if len(list(m.keys())) > 1 and len(list(l.keys())) > 1:
    ejercicio_1_solucion = cubo.query(
        m[list(m.keys())[5]],  
        levels=[l[list(l.keys())[2]]]  
    )
    
    print(f"📊 Medida usada: {list(m.keys())[5]}")
    print(f"🎯 Nivel usado: {list(l.keys())[2]}")
    display(ejercicio_1_solucion)
else:
    print("⚠️ Recursos limitados para este ejercicio")

🎯 EJERCICIO 1: Crea una consulta que agrupe por una dimensión diferente
💪 Objetivo: Practicar la sintaxis básica de consulta MDX
📝 Instrucciones:
   1. Selecciona una medida disponible
   2. Selecciona un nivel para agrupar (diferente a los ejemplos)
   3. Ejecuta la consulta
   4. Interpreta los resultados

📋 Recursos disponibles:
  Medidas: ['NUM_REGS_GRANO_MOV.MEAN', 'ID_CAMPUS_CENTRO.SUM', 'FLG_TRASLADO_MISMO_ESTUDIO.MEAN']...
  Niveles: [('CursosAcademicos', 'NOMBRE_CURSO_ACADEMICO', 'NOMBRE_CURSO_ACADEMICO'), ('Sexo', 'FECHA_CARGA', 'FECHA_CARGA'), ('Sexo', 'NOMBRE_SEXO', 'NOMBRE_SEXO')]...

💡 Pista: ejercicio_1 = cubo.query(m['tu_medida'], levels=[l['tu_nivel']])
📝 Escribe tu código aquí:


✅ EJEMPLO DE SOLUCIÓN:
📊 Medida usada: CREDITOS.MEAN
🎯 Nivel usado: ('Sexo', 'NOMBRE_SEXO', 'NOMBRE_SEXO')


,CREDITOS.MEAN
NOMBRE_SEXO,
Hombre,7.33
Mujer,7.49


In [38]:
# 🔰 EJERCICIO 2: Aplicar un filtro básico
print("🎯 EJERCICIO 2: Crea una consulta con filtro")
print("💪 Objetivo: Practicar filtros en consultas MDX")
print("📝 Desafío: Filtra los datos para mostrar solo un subconjunto")

print("\n💡 Pistas:")
print("   - Usa filter=nivel == 'valor' para igualdad")
print("   - Usa filter=medida > numero para condiciones numéricas")
print("   - Combina con & para múltiples condiciones")

print("\n📝 Tu turno: Escribe una consulta con filtro\n")

# TU CÓDIGO AQUÍ:
# ejercicio_2 = cubo.query(..., filter=...)

print("\n" + "="*50)
print("✅ EJEMPLO DE SOLUCIÓN:")

# Filtro numérico simple
ejercicio_2_solucion = cubo.query(
    m[medida_count],
    levels=[nivel_para_agrupar],
    filter=m[medida_count] >= 1  # Solo grupos con al menos 1 registro
)

print("🎯 Filtro aplicado: Conteo >= 1")
display(ejercicio_2_solucion)
print(f"📊 Grupos que cumplen el filtro: {len(ejercicio_2_solucion)}")

🎯 EJERCICIO 2: Crea una consulta con filtro
💪 Objetivo: Practicar filtros en consultas MDX
📝 Desafío: Filtra los datos para mostrar solo un subconjunto

💡 Pistas:
   - Usa filter=nivel == 'valor' para igualdad
   - Usa filter=medida > numero para condiciones numéricas
   - Combina con & para múltiples condiciones

📝 Tu turno: Escribe una consulta con filtro


✅ EJEMPLO DE SOLUCIÓN:
🎯 Filtro aplicado: Conteo >= 1


,contributors.COUNT
NOMBRE_CURSO_ACADEMICO,
2010/2011,28
2011/2012,31
2012/2013,38
2013/2014,50
2014/2015,27
2015/2016,36
2016/2017,28
2017/2018,29
2018/2019,30


📊 Grupos que cumplen el filtro: 15


### 🚀 Ejercicios Avanzados

In [40]:
# 🚀 EJERCICIO 3: Consulta multidimensional compleja
print("🎯 EJERCICIO 3: Análisis multidimensional completo")
print("💪 Desafío: Combina múltiples medidas, niveles y filtros")
print("📝 Requisitos:")
print("   1. Al menos 2 medidas diferentes")
print("   2. Al menos 1 nivel de agrupación")
print("   3. Un filtro para limitar los datos")
print("   4. Ordenamiento de resultados")

print("\n🎯 Meta: Crear un análisis útil para toma de decisiones")
print("📝 Tu código aquí:\n")

# TU CÓDIGO AQUÍ:
# ejercicio_3 = cubo.query(...)
# ejercicio_3_ordenado = ejercicio_3.sort_values(...)

print("\n" + "="*50)
print("✅ SOLUCIÓN GUIADA:")

print("📝 Paso 1: Consulta multidimensional")
ejercicio_3_base = cubo.query(
    m[medida_count],
    *[m[medida] for medida in medidas_seleccionadas[:2]],
    levels=[nivel_para_agrupar],
    filter=m[medida_count] > 0
)

print("📝 Paso 2: Ordenamiento por importancia")
ejercicio_3_final = ejercicio_3_base.sort_values(
    by=ejercicio_3_base.columns[0],
    ascending=False
).head(5)

print("📝 Paso 3: Análisis de resultados")
display(ejercicio_3_final)

print("\n🎯 Interpretación del análisis:")
print("  - ✅ Medidas múltiples: Vista panorámica")
print("  - ✅ Filtro aplicado: Solo datos relevantes")
print("  - ✅ Ordenamiento: Priorización por importancia")
print("  - ✅ Limitación: Focus en Top 5")

🎯 EJERCICIO 3: Análisis multidimensional completo
💪 Desafío: Combina múltiples medidas, niveles y filtros
📝 Requisitos:
   1. Al menos 2 medidas diferentes
   2. Al menos 1 nivel de agrupación
   3. Un filtro para limitar los datos
   4. Ordenamiento de resultados

🎯 Meta: Crear un análisis útil para toma de decisiones
📝 Tu código aquí:


✅ SOLUCIÓN GUIADA:
📝 Paso 1: Consulta multidimensional
📝 Paso 2: Ordenamiento por importancia
📝 Paso 3: Análisis de resultados


,contributors.COUNT,NUM_REGS_GRANO_MOV.MEAN,ID_CAMPUS_CENTRO.SUM
NOMBRE_CURSO_ACADEMICO,,,
2013/2014,50,<NA>,<NA>
2012/2013,38,<NA>,<NA>
2024/2025,38,<NA>,<NA>
2022/2023,38,<NA>,<NA>
2015/2016,36,<NA>,<NA>



🎯 Interpretación del análisis:
  - ✅ Medidas múltiples: Vista panorámica
  - ✅ Filtro aplicado: Solo datos relevantes
  - ✅ Ordenamiento: Priorización por importancia
  - ✅ Limitación: Focus en Top 5


## 🛠️ TROUBLESHOOTING: Errores Comunes y Soluciones

In [ ]:
# 🔧 FUNCIÓN DE DIAGNÓSTICO AVANZADA
def diagnosticar_consulta_mdx(cubo, medidas_test=None, niveles_test=None):
    """
    Función completa de diagnóstico para consultas MDX
    """
    print("🔍 DIAGNÓSTICO COMPLETO DE CONSULTA MDX\n")
    
    # Estado del cubo
    print("🧊 ESTADO DEL CUBO:")
    print(f"  ✅ Nombre: {cubo.name}")
    print(f"  📊 Medidas disponibles: {len(cubo.measures)}")
    print(f"  📐 Niveles disponibles: {len(cubo.levels)}")
    print(f"  🏗️ Jerarquías: {len(cubo.hierarchies)}")
    
    # Verificar conectividad básica
    try:
        test_query = cubo.query(list(cubo.measures.values())[0])
        print(f"  ✅ Conectividad: OK (registros: {test_query.iloc[0,0]})")
    except Exception as e:
        print(f"  ❌ Error de conectividad: {e}")
    
    # Validar medidas específicas
    if medidas_test:
        print("\n📊 VALIDACIÓN DE MEDIDAS:")
        for medida in medidas_test:
            if medida in cubo.measures:
                print(f"  ✅ {medida} - VÁLIDA")
            else:
                print(f"  ❌ {medida} - NO ENCONTRADA")
                similares = [m for m in cubo.measures.keys() if medida.upper() in m.upper()]
                if similares:
                    print(f"      💡 Similares: {similares[:3]}")
    
    # Validar niveles específicos
    if niveles_test:
        print("\n📐 VALIDACIÓN DE NIVELES:")
        for nivel in niveles_test:
            if nivel in cubo.levels:
                print(f"  ✅ {nivel} - VÁLIDO")
            else:
                print(f"  ❌ {nivel} - NO ENCONTRADO")
                similares = [l for l in cubo.levels.keys() if nivel.upper() in str(l).upper()]
                if similares:
                    print(f"      💡 Similares: {similares[:3]}")
    
    # Recursos recomendados
    print("\n🎯 RECURSOS RECOMENDADOS:")
    print("📊 Mejores medidas para empezar:")
    medidas_count = [m for m in cubo.measures.keys() if 'COUNT' in m.upper()][:3]
    for medida in medidas_count:
        print(f"  🔢 {medida}")
    
    print("\n📐 Mejores niveles para agrupar:")
    primeros_niveles = list(cubo.levels.keys())[:3]
    for nivel in primeros_niveles:
        print(f"  🎯 {nivel}")
    
    print("\n💡 CONSEJOS DE TROUBLESHOOTING:")
    print("  1. Siempre valida nombres exactos antes de consultar")
    print("  2. Usa medidas COUNT para consultas básicas")
    print("  3. Verifica tipos de datos al aplicar filtros")
    print("  4. Maneja resultados vacíos con try/except")

# Ejecutar diagnóstico del cubo actual
diagnosticar_consulta_mdx(cubo, ["COUNT", "MEAN"], ["SEMESTRE", "CARRERA"])

In [ ]:
# 🚨 PATRONES DE ERRORES COMUNES Y SOLUCIONES
print("🚨 ERRORES COMUNES EN CONSULTAS MDX Y SUS SOLUCIONES\n")

print("❌ ERROR 1: 'Medida no encontrada'")
print("💡 SOLUCIÓN: Verificar nombres exactos")
print("   ✅ Correcto: m['contributors.COUNT']")
print("   ❌ Incorrecto: m['count'] o m['COUNT']")

print("\n❌ ERROR 2: 'Nivel no válido'")
print("💡 SOLUCIÓN: Explorar structure del cubo")
print("   ✅ Usar: list(cubo.levels.keys()) para ver opciones")
print("   ✅ Formato: ('tabla', 'columna', 'columna')")

print("\n❌ ERROR 3: 'Filtro incompatible'")
print("💡 SOLUCIÓN: Verificar tipos de datos")
print("   ✅ Texto: nivel == 'string'")
print("   ✅ Número: medida > numero")
print("   ✅ Combinado: (condicion1) & (condicion2)")

print("\n❌ ERROR 4: 'Resultado vacío'")
print("💡 SOLUCIÓN: Validar datos y condiciones")
print("   ✅ Verificar que existen datos en las tablas")
print("   ✅ Relajar filtros demasiado restrictivos")
print("   ✅ Usar try/except para manejo elegante")

print("\n✅ MEJORES PRÁCTICAS:")
print("  1. 🔍 Siempre explorar estructura antes de consultar")
print("  2. 🧪 Probar consultas simples antes de complejas")
print("  3. 📊 Validar resultados con consultas conocidas")
print("  4. 🛡️ Implementar manejo de errores robusto")
print("  5. 📝 Documentar consultas complejas para reuso")

## 📚 RESUMEN Y PRÓXIMOS PASOS

### ✅ Lo que has dominado

**🔰 Fundamentos MDX:**
- ✅ Estructura básica: `cubo.query(medidas, levels=niveles, filter=filtro)`
- ✅ Consultas simples con una dimensión
- ✅ Consultas con múltiples medidas
- ✅ Aplicación de filtros básicos y complejos

**🚀 Técnicas avanzadas:**
- ✅ Análisis bidimensional (cruce de variables)
- ✅ Ordenamiento y Top N con pandas
- ✅ Interpretación de resultados multivariados
- ✅ Troubleshooting y validación de consultas

### 🎯 Conceptos clave consolidados

1. **Sintaxis MDX** - Estructura estándar para todas las consultas
2. **Medidas vs Niveles** - Diferencia entre "qué medir" y "cómo agrupar"
3. **Filtros estratégicos** - Cuando y cómo limitar datos
4. **Pipeline MDX+Pandas** - Combinar consulta + manipulación
5. **Interpretación de resultados** - Leer y analizar matrices multidimensionales

### 🚀 Preparación para Notebook 03: Cubos Multidimensionales

Con las bases sólidas de consultas MDX, estarás listo para:
- 🧊 **Diseño de cubos complejos** - Múltiples tablas de hechos
- 🏗️ **Jerarquías anidadas** - Dimensiones con múltiples niveles
- 🔄 **Drill-down y Roll-up** - Navegación entre niveles
- 📊 **Medidas calculadas avanzadas** - Fórmulas personalizadas
- 🎨 **Visualizaciones sofisticadas** - Dashboards interactivos

### 💪 Ejercicios para consolidar

**Antes del siguiente notebook, practica:**
1. Crear 5 consultas MDX diferentes con tus propios datos
2. Combinar filtros complejos con múltiples condiciones
3. Analizar resultados bidimensionales e interpretar patrones
4. Usar la función de diagnóstico para resolver errores

### 🎉 ¡Felicitaciones!

Has completado exitosamente los **fundamentos de consultas MDX**. Ahora tienes las herramientas esenciales para realizar análisis multidimensional básico. En el próximo notebook, aplicaremos estos conocimientos para crear cubos OLAP sofisticados que resuelvan problemas de negocio reales.

---

**🔗 Continuación**: `03_cubos_multidimensionales.ipynb` - Donde llevarás tus habilidades MDX al siguiente nivel.